# Trabalho 3: Avaliando sistemas de IR

Neste trabalho, o aluno irá explorar o problema de otimização de sistemas de IR através da análise de métricas de avaliação populares para esse tipo de sistema.

Para isso, vamos usar a base CFC (com artigos sobre fibrose cística), a qual encontra-se nesta pasta.

Observação: a idéia aqui é usar o jupyter para mostrar a evolução dos teus experimentos. Então use ele adequadamente. Ou seja, não altere funções do começo para reexecutar algo que foi feito lá embaixo. Neste caso, sobreescreva a função lá embaixo. A leitura do notebook tem que ser feita sequencialmente para ficar fácil, ok?


## Passo 1 - Normalização

Primeira coisa que você vai precisar fazer é ler os arquivos do CFC. Essa é uma parte meio braçal mesmo, mas não tem como escapar dela. Então você terá que criar funções que consigam ler cada arquivo do CFC e parsear eles.


In [1]:
#import ipdb;ipdb.set_trace()

import pandas as pd
import json

FILES = ['cfc/cf74', 'cfc/cf75', 'cfc/cf76', 'cfc/cf77', 'cfc/cf78', 'cfc/cf79']
matrixTotal = []

def insert(i, value, vector):
    if(len(vector) <= i):
        vector.insert(i,value)
    else:
        vector[i] += ' ' + value


def constructVector(vector, line, cod):  
    if(cod == 'PN'):
        insert(0, line[3:], vector)
        
    elif(cod == 'RN'):
        insert(1, line[3:], vector)
        
    elif(cod == 'AN'):
        insert(2, line[3:], vector)
        
    elif(cod == 'AU'):
        insert(3, line[3:], vector)

    elif(cod == 'TI'):
        insert(4, line[3:], vector)
        
    elif(cod == 'SO'):
        insert(5, line[3:], vector)
        
    elif(cod == 'MJ'):
        insert(6, line[3:], vector)
    
    elif(cod == 'MN'):
        insert(7, line[3:], vector)
        
    elif((cod == 'AB') or (cod == 'EX')):
        insert(8, line[3:], vector)
        
    elif(cod == 'RF'):
        insert(9, line[3:], vector)
        
    elif(cod == 'CT'):
        insert(10, line[3:], vector)
    else:
        pass
    

def fileToMatrix(file):
    matrix = []
    with open(file, 'r', encoding = 'iso-8859-1') as fp:
        vector = []
        line = fp.readline()
        cod = line[:2]
        constructVector(vector, line.strip().replace("  ", ""), cod)

        while line:
            line = fp.readline()
            if not((line[:2]).isspace()):
                cod = line[:2]
            if((line == "\n") and (len(vector) > 2)):
                matrix.append(vector)
                vector = []
            constructVector(vector, line.strip().replace("  ", ""), cod)    
    
    fp.close()
    return matrix

        
for file in FILES:
    matrixTotal.extend(fileToMatrix(file))  
    
    
pd.DataFrame(matrixTotal)    

#m = fileToMatrix('cfc/cf74')


,0,1,2,3,4,5,6,7,8,9,10
0,74001,00001,75051687,Hoiby-N.Jacobsen-L.Jorgensen-B-A.Lykkegaard-E....,Pseudomonas aeruginosa infection in cystic fib...,Acta-Paediatr-Scand. 1974 Nov. 63(6). P 843-8.,CYSTIC-FIBROSIS: co.PSEUDOMONAS-AERUGINOSA: im...,ADOLESCENCE.BLOOD-PROTEINS: me.CHILD.CHILD-PRE...,The significance of Pseudomonas aeruginosa inf...,001 BELFRAGE SACTA MED SCAND SUPPL 173 5 963 ...,1 HOIBY N SCAND J RESPIR DIS5638 975 OIBY N AC...
1,74002,00002,74260154,Rossiter-M-A.Barrowman-J-A.Dand-A.Wharton-B-A.,Amylase content of mixed saliva in children.,Acta-Paediatr-Scand. 1974 May. 63(3). P 389-92.,SALIVA: en.AMYLASES: me.NUTRITION-DISORDERS: e...,INFANT.CHILD-PRESCHOOL.CHILD.ADOLESCENCE.HUMAN...,Salivary amylase levels were determined in nor...,001 ANDERSEN DH J PEDIATR 30 564 947 BARBEZAT...,1 ALHADEFF JA CLIN GENET1063 976 ASZLO AACTA P...
2,74003,00003,74252221,Kollberg-H.Ekbohm-G.,A clinical study of the diagnosis of cystic fi...,Acta-Paediatr-Scand. 1974 May. 63(3). P 411-7.,CYSTIC-FIBROSIS: di.NAILS: an.SODIUM: an.,INFANT-NEWBORN.INFANT.CHILD-PRESCHOOL.CHILD.AD...,This article reports on the possibility of usi...,001 BOWEN HJM ADV ACTIVATION ANALYS1 101 969 ...,1 KOLLBERG HSCAND J CLIN LAB INVEST 3517 975 A...
3,74004,00004,74252220,Kollberg-H.Landstrom-O.,A methodological study of the diagnosis of cys...,Acta-Paediatr-Scand. 1974 May. 63(3). P 405-10.,CYSTIC-FIBROSIS: di.NAILS: an.SODIUM: an.,CHILD-PRESCHOOL.HUMAN.CYSTIC-FIBROSIS: me.ACTI...,Instrumental neutron activation analysis (INAA...,001 ANDERSEN GH TRANS AM NUCL SOC 1057 967 AN...,1 HELLSING KSCAND J CLIN LAB INVEST 33 333 974...
4,74005,00005,75051680,Lindberg-T.,Proteolytic activity in duodenal juice in infa...,Acta-Paediatr-Scand. 1974 Nov. 63(6). P 805-8.,DUODENUM: en.INTESTINAL-SECRETIONS: en.PEPTIDE...,ADULT.AGE-FACTORS.CASEINS: me.CELIAC-DISEASE: ...,The capacity of duodenal juice to hydrolyse ca...,001 BERGSTROM K SCAND J GASTROENTEROL5 533 970...,1 BROCK JHIMMUNOLOGY32 207 977 CHMITZ J GASTRO...
5,74006,00006,75014083,Hoiby-N.Mathiesen-L.,Pseudomonas aeruginosa infection in cystic fib...,Acta-Pathol-Microbiol-Scand [B]. 1974 Aug. 82(...,B-LYMPHOCYTES: im.CYSTIC-FIBROSIS: im.IMMUNITY...,ADOLESCENCE.ANTIBODIES-BACTERIAL.ANTIGENS-BACT...,The influence of chronic Ps. aeruginosa infect...,001 AIUTI F INFECT IMMUN 8 110 973 AIUTI F CL...,1 HOIBY N ACTA PATH MICROBIOL SCAND (B) 82 551...
6,74007,00007,75014084,Hoiby-N.,Pseudomonas aeruginosa infection in cystic fib...,Acta-Pathol-Microbiol-Scand [B]. 1974 Aug. 82(...,CYSTIC-FIBROSIS: im.IMMUNITY.PSEUDOMONAS-AERUG...,ANTIBODIES-BACTERIAL.ANTIGENS-BACTERIAL.CYSTIC...,The occurrence of Pseudomonas aeruginosa in th...,001 BURNS MWLANCET 1 270 968 BURNS MWBR MED J...,1 HOIBY N ACTA PATH MICROBIOL SCAND (B) 82 541...
7,74008,00008,75009735,Hoiby-N.,Epidemiological investigations of the respirat...,Acta-Pathol-Microbiol-Scand [B]. 1974 Aug. 82(...,CYSTIC-FIBROSIS: mi.BACTERIA: ip.BACTERIA: ip....,ADOLESCENCE.CHILD.CHILD-PRESCHOOL.CYSTIC-FIBRO...,Seventy patients with cystic fibrosis treated ...,001 BURNS MWLANCET 1 270 968 COWAN STMANUAL F...,1 HOIBY N ACTA PATH MICROBIOL SCAND (B) 82 559...
8,74009,00009,74087894,Franklin-J-L.Asquith-P.Rosenberg-I-H.,The occurrence of cystic fibrosis and celiac s...,Am-J-Dig-Dis. 1974 Feb. 19(2). P 149-55.,CYSTIC-FIBROSIS: fg.CELIAC-DISEASE: fg.,"ADULT.BIOPSY.CELIAC-DISEASE: co, fg.CHILD. TIC...",A family is presented in which celiac sprue an...,001 HIDE DW ARCH DIS CHILD44 533 969 ANTONOWI...,1 SAFWENBERG JTISSUE ANTIGENS 10 287 977
9,74010,00010,74154352,Burnell-R-H.Robertson-E-F.,Cystic fibrosis in a patient with Kartagener s...,Am-J-Dis-Child. 1974 May. 127(5). P 746-7.,CYSTIC-FIBROSIS: co.KARTAGENER-TRIAD: co.,CASE-REPORT.CHLORIDES: an.HUMAN.INFANT.LUNG: r...,A patient exhibited the features of both Karta...,001 KARTAGENER MBEITR KLIN TUBERK 83 489 933 ...,1 BOCHKOVA DN GENETIKA (SOVIET GENETICS)11 154...


In [ ]:
# fazendo parse do arquivo de query
import numpy as np

cfquery = 'cfc/cfquery'
queryFile = []
weight = [1, 1, 1, 1] # peso, repectivamente, referente aos avaliadores: REW, REW colleagues, REW post-doctorates e JBW

def scoreCalc(weigth, v):
    values = [int(digit) for digit in v]
    weighted_avg = np.average(values, weights=weight)
    return weighted_avg

def insertQ(i, value, vector):
    if(len(vector) <= i):
        vector.insert(i,value)
    else:
        if(type(value) is list):
            print(vector[i])
            vector[i].extend(value)
        else:
            vector[i] += ' ' + value

def constructQueryVector(vector, line, cod):  
    if(cod == 'QN'):
        insertQ(0, line, vector)
        
    elif(cod == 'QU'):
        insertQ(1, line, vector)
        
    elif(cod == 'NR'):
        insertQ(2, line, vector)
        
    elif(cod == 'RD'):
        v = line.split("  ")
        for i, result in enumerate(v):
            v1 = result.split(" ")
            score = scoreCalc(weight, v1[1])
            v1.insert(1,score)
            v.insert(i, v1)
        
        insertQ(3, v, vector)
    else:
        pass

def queryMatrix(file):
    matrix = []
    with open(file, 'r', encoding = 'iso-8859-1') as fp:
        vector = []
        line = fp.readline()
        cod = line[:2]
        constructQueryVector(vector, line[3:].strip(), cod)
        
        while line:
            line = fp.readline()
            
            if not((line[:2]).isspace()):
                cod = line[:2]
                line = line[3:]
        
            if(line.isspace()):
                matrix.append(vector)
                vector = []
            constructQueryVector(vector, line.strip(), cod)    
    
    fp.close()
    return matrix

queryFile.extend(queryMatrix(cfquery))
pd.DataFrame(queryFile)    
print("oi")

## Passo 2 - Primeira indexação

Depois de parsear os arquivos, agora é hora de indexar os dados. Ao invés de criar o código do zero, vamos usar o Whoosh, que é uma implementação em python inspirada no Lucene. A documentação pode ser encontrada em https://whoosh.readthedocs.io/en/latest/index.html.

Nesta questão de indexação, temos que fazer algumas escolhas: 
* Como fazer o processo de tokenização? 
* Stemmizar ou não stemmizar, eis a questão...
* Quais campos são úteis para indexação?

Neste momento, você vai tomar suas decisões iniciais. A ideia é você testar, depois, outras soluções para verificar quais tiveram os melhores resultados, entendeu? Então não esqueça de documentar, aqui, qual a sua decisão inicial e depois ir explicando ao longo do notebook os experimentos que está fazendo.

In [17]:
from whoosh.fields import Schema, TEXT, KEYWORD, ID, STORED,NUMERIC
from whoosh.analysis import StemmingAnalyzer,StandardAnalyzer
from whoosh import index
import os, os.path, shutil

schema = None

def creatIndex():
    #schema definition
    global schema
    schema = Schema(pn=NUMERIC(stored=True), #paper_number
                    rn=NUMERIC(stored=True), #record_number
                    au=TEXT(stored=True), #authors
                    ti=TEXT(analyzer=StemmingAnalyzer(stoplist=None),stored=True), #title
                    mj=TEXT(stored=True), #major_subjects
                    mn=TEXT(stored=True), #minor_subjects
                    ab=TEXT(analyzer=StemmingAnalyzer(stoplist=None),stored=True), #abstract or extract
                   )

    if not(os.path.exists("index")): # create an index directory
        os.mkdir("index") 
    # creating a index writer to add document 
    ix = index.create_in("index", schema)
    ix = index.open_dir("index") #open an existing index object
    writer = ix.writer() #create a writer object to add documents to the index

    for doc in matrixTotal:
        writer.add_document(pn=doc[0], #add documents to the index
                           rn=doc[1],
                           au=doc[3],
                           ti=doc[4],
                           mj=doc[6],
                           mn=doc[7],
                           ab=doc[8])
    writer.commit()


creatIndex()

## Passo 3 - Calculando as métricas de avaliação

Para avaliar a tua decisão, vamos usar como métricas a precisão, recall e f-measure. Faça uma variação da precisão e recall como P@n e R@n, pois serão úteis para os gráficos que serão gerados. 

### 3.1 - Fazendo as queries

In [13]:
# normalizando as query para poder usar no whoosh
import spacy
from spacy.lang.pt.stop_words import STOP_WORDS
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer('english')
nlp = spacy.load('en')


def norma(text):
    text = nlp(text.lower())
    termos = [token for token in text if not (token.is_stop or token.pos_ == 'PUNCT' or token.pos_ == 'PRON' or token.pos_ == 'CCONJ' or token.pos_ == 'DET')]
    texto = ''
    for term in termos:
        texto += ' ' + stemmer.stem(term.orth_) 
    return texto


In [69]:
from whoosh.qparser import *
from whoosh import scoring

queryTest = []

#query = norma(queryFile[0][1])

def makeQuery(query):
    og = OrGroup.factory(0.9)
    parser = MultifieldParser(["ti", "ab", "mj", "mn"], schema=schema, group=og) 
    result=parser.parse(query)   
    return result
    
def storeQueryResult():    
    ix = index.open_dir("index") 

    for result in queryFile:
        v = []
        v.append(result[0]) #question_number
        query = result[1] #query
        
        with ix.searcher() as searcher:
            result = makeQuery(norma(query))
            results=searcher.search(result, limit=None)#The Results object acts like a list of the matched documents.
            v.append(len(results)) 
            docs = []
            for doc in results:
                docs.append(doc.get('rn'))
            v.append(docs)
        
        queryTest.append(v)
        
storeQueryResult()
pd.DataFrame(queryTest) 

,0,1,2
0,00001,944,"[00533, 00754, 00052, 00139, 00957, 00484, 003..."
1,00002,671,"[00754, 00614, 00761, 00400, 00160, 00137, 002..."
2,00003,1007,"[00633, 01206, 00137, 00959, 00269, 00856, 007..."
3,00004,503,"[00715, 00604, 00725, 01039, 00374, 00876, 009..."
4,00005,501,"[00499, 00754, 00498, 00151, 00975, 00592, 002..."
5,00006,979,"[00980, 00031, 00496, 00047, 00441, 00593, 005..."
6,00007,1023,"[00754, 00633, 00856, 00151, 00371, 01064, 007..."
7,00008,779,"[00166, 00011, 00439, 00614, 01173, 00589, 001..."
8,00009,478,"[00339, 00165, 00581, 00545, 00720, 00870, 004..."
9,00010,971,"[00581, 01218, 00676, 00157, 00878, 00165, 004..."


### 3.2 - Análise

In [71]:

def analysis(queryFile, queryTest):
    for query in queryFile:
        relevantDocs = query[3].split(" ")
        print(relevantDocs)
        
analysis(queryFile, queryTest)

['139', '1222', '', '151', '2211', '', '166', '0001', '', '311', '0001', '', '370', '1010', '', '392', '0001', '', '439', '0001', '440', '0011', '', '441', '2122', '', '454', '0100', '', '461', '1121', '', '502', '0002', '', '503', '1000', '', '505', '0001', '520', '1010', '', '522', '0001', '', '526', '1011', '', '527', '1001', '', '533', '2222', '', '593', '1000', '', '619', '0001', '737', '0100', '', '742', '0100', '', '789', '0001', '', '827', '0010', '', '835', '0001', '', '861', '0001', '', '875', '1121', '891', '0001', '', '921', '1010', '', '922', '1010', '1175', '0100', '1185', '0001', '1222', '0001']
['169', '1000', '', '434', '1001', '', '454', '0100', '', '498', '1000', '', '499', '1000', '', '592', '0002', '', '875', '1011']
['23', '1000', '', '', '40', '0010', '', '139', '2122', '', '190', '0001', '', '221', '0001', '', '246', '0001', '', '309', '1000', '311', '0011', '', '325', '0001', '', '345', '0010', '', '347', '0010', '', '356', '0010', '', '370', '1001', '', '374',

## Passo 4 - Gráficos do experimento 1

Apresentar agora os gráficos do teu experimento. Você deve gerar um gráfico PxR, como visto em sala. Para gerar gráficos, você pode usar o matplotlib. Se quiser usar outra biblioteca que ache mais fácil, sem problemas!

In [ ]:
# códigos...

## Passo 5 - Demais experimentos

Agora você irá testar novas configurações de campos e configurações para tentar encontrar qual a que te dá melhores resultados. Mostrar, como um relatório, a evolução do trabalho. Ou seja, mostrar como o trabalho foi evoluindo para você alcançar o melhor resultado (quais modificações foram feitas, como cada modificação influenciou nas métricas, etc).

In [ ]:
# códigos...

## Passo 6 - Quantos resultados eu devo voltar para o usuário?

Um dos problemas de sistemas de IR é determinar quantos resultados você deve retornar ao usuário. Uma forma de entender o comportamento do sistema e qual o corte ideal na lista resultante é através do uso de curvas ROC. Assim, plote a curva ROC do teu sistema e determine, através da análise da curva, qual o ponto ideal de corte para o teu sistema.

In [ ]:
# códigos...